# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 

# Import API key
from api_keys import g_key

In [2]:
#Loaded the csv exported in Part I to a DataFrame
weather_data = pd.read_csv("weather_data.csv") 
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kalmar,56.66,16.36,62.01,100,84,6.93,SE,1594854789
1,ushuaia,-54.80,-68.30,28.40,86,1,5.82,AR,1594854790
2,punta arenas,-53.15,-70.92,33.80,86,75,5.82,CL,1594854126
3,ibb,13.97,44.18,70.18,63,100,0.76,YE,1594854790
4,pacific grove,36.62,-121.92,73.00,60,1,12.75,US,1594854214
...,...,...,...,...,...,...,...,...,...
560,port macquarie,-31.43,152.92,59.00,64,26,7.00,AU,1594854902
561,great yarmouth,52.61,1.73,59.00,99,100,9.93,GB,1594854902
562,khuzhir,53.19,107.34,57.60,81,100,12.41,RU,1594854903
563,geraldton,-28.77,114.60,57.20,87,90,10.29,AU,1594854860


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#declared lat and long as location and set humidity as the "weight" inside gmaps 
locations = weather_data[["Lat", "Lng"]]
fig = gmaps.figure()
Humidity = weather_data["Humidity"].astype(float)

In [4]:
#heat layers declared as the heat map layer, added heatmap layer 
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness
#numbers changed from celsius to farenheight 
weather_criteria = pd.DataFrame(weather_data)
#temp_greater = weather_criteria.loc[weather_criteria["Max Temp"]>21.1111]
#temp_less = temp_greater.loc[temp_greater["Max Temp"]<29.4444]
#wind_low = temp_less.loc[temp_less["Wind Speed"]<10]
#temp = wind_low.loc[wind_low["Cloudiness"]==1]
weather_criteria.head()
fit_weather=weather_criteria.loc[(weather_criteria["Max Temp"]>80)&(weather_criteria["Max Temp"]<95)&(weather_criteria["Wind Speed"]<10)&(weather_criteria["Cloudiness"]==1)].dropna()
fit_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,jamestown,42.10,-79.24,82.99,47,1,4.70,US,1594854793
123,erzin,36.96,36.20,80.01,88,1,0.31,TR,1594854815
159,mayo,38.89,-76.51,88.00,54,1,8.05,US,1594854822
164,madison heights,42.49,-83.11,88.00,45,1,4.70,US,1594854821
218,broome,42.25,-75.83,81.00,44,1,4.09,US,1594854834
245,hamilton,39.18,-84.53,90.00,48,1,4.70,US,1594854839
247,middlebury,44.02,-73.17,80.60,32,1,5.82,US,1594854741
298,huntsville,34.73,-86.59,91.40,46,1,5.82,US,1594854849
343,morristown,36.21,-83.29,93.99,34,1,0.78,US,1594854603
380,saint anthony,45.02,-93.22,82.40,28,1,6.93,US,1594854859


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df=fit_weather[["City","Lat","Lng","Country"]].copy()
hotel_df["Hotel Name"]=""
hotel_df

,City,Lat,Lng,Country,Hotel Name
20,jamestown,42.10,-79.24,US,
123,erzin,36.96,36.20,TR,
159,mayo,38.89,-76.51,US,
164,madison heights,42.49,-83.11,US,
218,broome,42.25,-75.83,US,
245,hamilton,39.18,-84.53,US,
247,middlebury,44.02,-73.17,US,
298,huntsville,34.73,-86.59,US,
343,morristown,36.21,-83.29,US,
380,saint anthony,45.02,-93.22,US,


In [14]:
# geocoordinates
#target_coordinates = "43.6187102, -116.2146068"
#target_search = "Chinese"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hoteldata=response.json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hoteldata["results"][0]["name"]
    except(KeyError,IndexError):
        print("Got to the next hotel")
print(json.dumps(hoteldata,indent=4,sort_keys=True))
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 30.49439599999999,
                    "lng": -92.442698
                },
                "viewport": {
                    "northeast": {
                        "lat": 30.4960780302915,
                        "lng": -92.44118336970848
                    },
                    "southwest": {
                        "lat": 30.4933800697085,
                        "lng": -92.4438813302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f36c805fb60bd62220ab6a6ecbc1526b5480258d",
            "name": "Holiday Inn Express Eunice",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1250,
     

,City,Lat,Lng,Country,Hotel Name
20,jamestown,42.10,-79.24,US,Hampton Inn & Suites Jamestown
123,erzin,36.96,36.20,TR,Artemis Otel
159,mayo,38.89,-76.51,US,Resorts Resource Group International
164,madison heights,42.49,-83.11,US,Red Roof Inn Detroit - Royal Oak/Madison Heights
218,broome,42.25,-75.83,US,Chenango Valley State Park
245,hamilton,39.18,-84.53,US,Six Acres Bed & Breakfast
247,middlebury,44.02,-73.17,US,Courtyard by Marriott Middlebury
298,huntsville,34.73,-86.59,US,Embassy Suites by Hilton Huntsville Hotel & Spa
343,morristown,36.21,-83.29,US,Days Inn by Wyndham Morristown
380,saint anthony,45.02,-93.22,US,Holiday Inn Express Roseville-St. Paul


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
marker=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))